In [3]:

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')


import govcxanalyzer.emotion_text_classifier as et

classifier = et.initialize_emotion_classifier(topk=3)

prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)






[[{'label': 'joy', 'score': 0.9959298968315125}, {'label': 'anger', 'score': 0.0018055237596854568}, {'label': 'love', 'score': 0.0009452480589970946}]]


In [68]:

import pandas as pd
dfnews = pd.read_csv('../data/sample-press-releases.csv').drop_duplicates(["title", "description"])

print(dfnews.info())
dfnews.sample(2)





<class 'pandas.core.frame.DataFrame'>
Index: 395 entries, 0 to 469
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        395 non-null    object
 1   description  300 non-null    object
 2   datetime     395 non-null    object
 3   category     395 non-null    object
 4   source       395 non-null    object
 5   link         395 non-null    object
dtypes: object(6)
memory usage: 21.6+ KB
None


,title,description,datetime,category,source,link
97,How phased retirement works,The ins and outs of a program that a lot of feds are curious about.,2023-08-17 15:00:00-04:00,Pay & Benefits,FederalNewsNetwork,http://www.govexec.com/pay-benefits/2023/08/how-phased-retirement-works/389498/
115,Military Survivor Annuity Program finally offers long-promised open season to choose new benefic...,Some retired service members say they are either outright denied or given an impossible runaroun...,2023-07-26 14:44:01-04:00,Pay & Benefits,FederalNewsNetwork,http://www.govexec.com/pay-benefits/2023/07/military-survivor-annuity-program-finally-offers-lon...


In [69]:
dfnews["text"] = dfnews["title"].map(str) + ": " + dfnews["description"].map(str)
textcol = "text"#"description"

dfcomments = et.make_dfonlycomments(dfnews, textcol=textcol)


395
395


In [70]:
predslist = et.get_text_emotion_preds(classifier, dfcomments, textcol=textcol)

emotions_df = et.turn_catpredlist_to_df(predslist)


395


In [71]:
emotions_df.sample(2)

,title,description,datetime,category,source,link,text,emot_topk,emotion_top,emotion_score
239,IRS announces the end of most surprise taxpayer visits,"The change, effective immediately, comes due to concern for the safety of IRS employees and to m...",2023-07-24 15:19:07-04:00,Workforce,GovExec,http://www.govexec.com/workforce/2023/07/irs-announces-end-most-surprise-taxpayer-visits/388778/,"IRS announces the end of most surprise taxpayer visits: The change, effective immediately, comes...","[[{'label': 'fear', 'score': 0.4957038462162018}, {'label': 'anger', 'score': 0.3930157124996185...",fear,0.495704
93,Solving the phased retirement puzzle,What can you expect in actual income?,2023-08-24 13:58:46-04:00,Pay & Benefits,FederalNewsNetwork,http://www.govexec.com/pay-benefits/2023/08/solving-phased-retirement-puzzle/389707/,Solving the phased retirement puzzle: What can you expect in actual income?,"[[{'label': 'joy', 'score': 0.8019924759864807}, {'label': 'anger', 'score': 0.11599348485469818...",joy,0.801992


In [72]:
et.get_emotions_by_groupcol(emotions_df, groupcol="category")


category,emotion_top,Defense,Management,Oversight,Pay & Benefits,Tech,Workforce
0,anger,19,17,22,8,32,19
1,fear,2,6,7,0,21,5
2,joy,20,23,14,34,85,18
3,love,0,0,1,0,0,0
4,sadness,9,7,6,7,4,8
5,surprise,0,0,0,1,0,0


In [73]:

et.create_group_counts_distribution(emotions_df, groupcol="category")

,category,# Responses,% Responses
0,Tech,142,36.0
1,Management,53,13.0
2,Pay & Benefits,50,13.0
3,Oversight,50,13.0
4,Defense,50,13.0
5,Workforce,50,13.0


In [74]:
# Filter Comments by Specific Emotions

from emotion_text_classifier import posemotions, negemotions

filtered_comments_pos = et.filtercommments(emotions_df, emotionlist=posemotions)
print(filtered_comments_pos.shape[0])
filtered_comments_neg = et.filtercommments(emotions_df, emotionlist=negemotions)
print(filtered_comments_neg.shape[0])


195
199


In [82]:
pd.set_option("max_colwidth", 200)

filtered_comments_neg[["title", "description",  "emotion_score", "link"]].head(15)

,title,description,emotion_score,link
262,Coronavirus Roundup: The Pandemic Public Health Emergency Is Over,There’s a lot to keep track of. Here’s a list of this week’s news updates and stories you may have missed.,0.998905,https://www.govexec.com/management/2023/05/coronavirus-roundup-pandemic-public-health-emergency-over/386288/
250,Coronavirus Roundup: The Future of the OSHA Standard to Protect Healthcare Workers from COVID is Unclear,There’s a lot to keep track of. Here’s a list of this week’s news updates and stories you may have missed.,0.997618,https://www.govexec.com/management/2023/05/coronavirus-roundup-future-osha-standard-protect-healthcare-workers-covid-unclear/386575/
224,Study: Mental health stigmas persist throughout the security clearance process,A new report from Leidos finds that mental health perceptions hurt pipelines of talent into the intelligence community.,0.996956,http://www.govexec.com/workforce/2023/08/study-mental-health-stigmas-persist-throughout-security-clearance-process/389377/
270,Top Postal Regulator Calls for Expanded Reach Amid Feud With DeJoy,"As USPS management says its regulator is standing in its way, the commission's chairman seeks to grow to keep pace with DeJoy's changes.",0.996145,https://www.govexec.com/oversight/2023/05/top-postal-regulator-calls-expanded-reach-amid-feud-dejoy/386649/
110,"Bill aims to reinstate tax deduction for fed union dues, and more",Pay and benefits updates you may have missed.,0.995622,http://www.govexec.com/pay-benefits/2023/08/bill-aims-reinstate-tax-deduction-fed-union-dues-and-more/389046/
357,Remote Work Has Been Controversial For Decades,Tom Shoop joins the podcast to discuss the history of federal telework policy.,0.995202,https://www.govexec.com/workforce/2023/05/remote-work-has-been-controversial-decades/386443/
251,It’s Time to Rethink Degree Requirements for Federal Contractors,Outdated mandates that job candidates have 4-year educational degrees create barriers to getting the best people on the job.,0.994452,https://www.govexec.com/management/2023/05/its-time-rethink-degree-requirements-federal-contractors/386471/
227,"In battle with federal employee union, Biden fights to preserve the debt ceiling",The Biden administration is arguing that federal workers do not have standing in the case and it should therefore be thrown out.,0.994254,http://www.govexec.com/workforce/2023/08/battle-federal-employee-union-biden-fights-preserve-debt-ceiling/389232/
144,"DeJoy's USPS overhaul plan might not be carried out well, watchdog finds","Postal Service management sharply critiques auditors' findings, saying GAO has a ""fundamental misunderstanding"" of the agency.",0.991808,http://www.govexec.com/oversight/2023/08/dejoys-usps-overhaul-risk-poor-implementation-gao-finds/389587/
208,Biden to reverse Trump decision to move Space Command to Alabama,The decision to keep the command at its current home in Colorado will surely enrage Alabama politicians.,0.989652,http://www.govexec.com/defense/2023/08/biden-reverse-trump-decision-move-space-command-alabama/389007/


In [103]:
# most complex emotions detected (with multiple emotions being detected)
for rec in filtered_comments_neg.tail(15).to_dict('records'):
    if str(rec["description"]).lower() == "nan": continue
    else:
        print(rec["title"])
        print(rec["link"])
        print(rec["description"])
        lol = rec["emot_topk"]
        for ee in lol:

            for e in ee:
                print(e["label"], ": " , e["score"])
        
        print("\n\n")

Women and LGBTQ+ veterans say VA facilities ‘weren’t built with us in mind’
http://www.govexec.com/defense/2023/08/women-and-lgbtq-veterans-say-va-facilities-werent-built-us-mind/389166/
Women veterans are less likely to go to Veterans Affairs hospitals for health care due to harassment, stigma and a lack of resources and services.
sadness :  0.4111778140068054
fear :  0.3208141624927521
anger :  0.23192889988422394



Senator blocks Marine confirmation, leaving Corps with an acting commandant
http://www.govexec.com/defense/2023/07/marines-without-confirmed-commandant-after-sen-tuberville-objects-motion/388365/
Gen. David Berger relinquished his position hours before the motion, handing authorities over to Assistant Commandant Gen. Eric Smith.
anger :  0.39218902587890625
sadness :  0.3362443447113037
fear :  0.1568712592124939



VA extends PACT Act filing deadline after website glitches
http://www.govexec.com/defense/2023/08/va-extends-pact-act-filing-deadline-after-website-glitches/

In [111]:
pd.set_option("max_colwidth", 300)
print("positive emotion scores")
filtered_comments_pos[["title", "description", "source", "emotion_score","category",]].head(20)

positive emotion scores


,title,description,source,emotion_score,category
269,"Biden and McCarthy Strike Positive Tone After Debt Limit Talks, But No Deal Yet","""We don’t have an agreement yet, but I did feel the discussion was productive in areas that we have differences of opinion,” McCarthy said Monday.",GovExec,0.998797,Oversight
381,Thank the People Who Make Sure You Get Your Benefits,Public Service Recognition Week is a good time to honor those who created and those who maintain the federal retirement system.,GovExec,0.997979,Pay & Benefits
252,Want More Good Ideas from Your Workers? Try Giving Them a Reward – and a Choice,A recent study found that offering workers a choice of what type of reward they would like for offering good suggestions increased the volume of submissions and their creativity too.,GovExec,0.997771,Management
16,"New Presidential Innovation Fellows will design lasting, impactful ... - GSA",NaN,GSA,0.997352,Tech
358,Could a New COVID Vaccine Offer Lifelong Protection?,"""We need a better vaccine, one that provides years of robust protection with fewer booster shots against a variety of SARS-CoV-2 strains.""",GovExec,0.996472,Workforce
223,The federal government’s primary internship program is getting its first update in a decade,The Office of Personnel Management will propose regulations Wednesday governing the federal government’s Pathways Programs to make it more appealing for potential applicants and easier for agencies to use as a recruitment tool.,GovExec,0.996332,Workforce
106,"The ‘future is at risk’ for government’s biggest civilian payroll provider, study finds","The National Finance Center — a part of the Agriculture Department — provides payroll services for over 170 agencies, but a lot of them aren’t happy with the services they’re getting.",FederalNewsNetwork,0.996271,Pay & Benefits
29,America needs trustworthy digital identity - FedScoop,NaN,FedScoop,0.996199,Tech
177,Data initiatives need to innovate on public engagement: Here’s how we can continue to improve,"COMMENTARY | Long-term, effective stakeholder engagement can lead to long-term success.",GovExec,0.996121,Tech
254,Passport Bottleneck Is Holding up International Travel by Americans Eager to See the World as COVID-19 Eases,Precautions taken in response to COVID-19 explain some but not all of the recent long delays in getting a passport.,GovExec,0.995864,Management
